In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import numpy as np
from src.config import *
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.metrics import roc_curve, auc
import json
from scipy.stats import mannwhitneyu
from scipy.stats import ttest_ind


In [4]:
from src.model_creation.CreateModelFunctions import fit_linear_model_and_get_coefficients_by_receptor as fit_model
from src.prediction_functions.MatrixMultiplicationMemoryEffectiveChunks import estimate_receptor_activity



### Calculate receptor confidence levels using MWU test

#### Read in data

In [5]:
lincs_consensus = pd.read_csv('data/lincs_consensus/high_quality/lm_all_pert_cell_liana.csv', index_col = 0) 
prior_knowledge = pd.read_csv('data/design_matrices/high_quality/all_pert_binary_liana.csv', index_col = 0)

In [6]:
receptor_activities = {}
# receptor activities were calculated before
for i in range(0, 5):
    receptor_activities[i] = pd.read_csv(f'results/confidence/receptor_activities_split_229_s0_{i}.csv', index_col =0)

### Calculate p-value

In [7]:
def create_negativ_and_positive_binary(binary):
    # print("Create positive value matrices for ROC curve calculation")

    if (binary == -1).any().any() == False:
        print('There are only positive perturbations, no negative matrix')
        negative_binary = pd.DataFrame()
    else:
        negative_binary = binary.replace({1:0})
        assert set(np.unique(negative_binary.values)) == {-1, 0}
        # delete only 0 rows
        s = negative_binary.sum() != 0
        negative_binary = negative_binary.loc[:, s.values]
        # change sign
        negative_binary = negative_binary.replace({-1:1})
        assert set(np.unique(negative_binary.values)) == {0, 1}

    if (binary == 1).any().any() == False:
        print('There are only negative perturbations, no positive matrix')
        positive_binary = pd.DataFrame()
    else: 
        positive_binary = binary.replace({-1:0})
        assert set(np.unique(positive_binary.values)) == {0, 1}
        # delete only 0 rows
        s = positive_binary.sum() != 0
        positive_binary = positive_binary.loc[:, s.values]

    return negative_binary, positive_binary

In [8]:
def mwu_test(receptor_binary, receptor_activity, side):
    if side == 'negative': alt = 'less'
    elif side == 'positive': alt = 'greater'
    pos = receptor_binary[receptor_binary == 1].index
    neg = receptor_binary[receptor_binary == 0].index
    pos = receptor_activity.loc[pos]
    neg = receptor_activity.loc[neg]
    if len(pos) >8: # under 8 it will be a recursion error, MWU test cannot be calculated
        u_stat, p_value = mannwhitneyu(pos.values, neg.values, alternative=alt, axis=0)
    else:
        u_stat, p_value = np.nan, 1
    return u_stat, p_value


In [9]:
def calculate_mwu_pvalues(binary, activities, side):
    receptors = list(set(activities.columns) & set(binary.columns))
    filtered_activities  = activities.loc[binary.index, receptors]
    filtered_binary = binary.loc[filtered_activities.index,receptors]

    mwu_stat = dict()
    mwu_pval = dict()
    for receptor in filtered_binary.columns:
        mwu_stat[receptor], mwu_pval[receptor] = mwu_test(filtered_binary.loc[:, receptor], filtered_activities.loc[:, receptor], side)
    return mwu_stat, mwu_pval


In [10]:
with open('results/confidence/test_indices_229_s0.json') as f:
    test_indices_dict = json.load(f)

In [11]:

def calculate_mwustatistics(activities, prior_knowledge):

    assert activities.shape[1] != 978
    negative_binary, positive_binary = create_negativ_and_positive_binary(prior_knowledge)
    results = {}
    if len(negative_binary) != 0:
        negative_results = {}
        negative_results['mwu_stat'], negative_results['mwu_pvalue'] = calculate_mwu_pvalues(negative_binary, activities, side = 'negative')
        results['negative_results'] = negative_results
    if len(positive_binary) != 0:
        positive_results = {}
        positive_results['mwu_stat'], positive_results['mwu_pvalue'] = calculate_mwu_pvalues(positive_binary, activities, side = 'positive')
        results['positive_results'] = positive_results
    # print('---- Done ----')
    return results

In [12]:
activities = receptor_activities[0]
prior_knowledge_temp = prior_knowledge.loc[test_indices_dict['0']]

In [13]:
# import sys
# sys.setrecursionlimit(4000)

In [14]:

all_results = {}
for i in tqdm(range(5)):
    results = calculate_mwustatistics(activities = receptor_activities[i], prior_knowledge = prior_knowledge.loc[test_indices_dict[str(i)]])
    all_results[i] = results

100%|██████████| 5/5 [00:15<00:00,  3.14s/it]


In [15]:
# get all receptors
all_receptor = []
for i in all_results.keys():
    for results in all_results[i].keys():
        mwustat = all_results[i][results]['mwu_stat']
        all_receptor.extend(list(mwustat.keys()))
all_receptor = set(all_receptor)

In [16]:
split_list = []
direction_list = []
receptor_list = []
metric_list = []
stat_list = []
for split, results in all_results.items():
    for direction, metrics in results.items():
        if direction in ['negative_results', 'positive_results']:
            for metric, values in metrics.items():
                    for receptor, stat in values.items():
                        split_list.append(split)
                        direction_list.append(direction)
                        metric_list.append(metric)
                        receptor_list.append(receptor)
                        stat_list.append(stat)


all_results_df = pd.DataFrame({
    'Split': split_list,
    'Direction': direction_list,
    'Receptor': receptor_list,
    'Metric': metric_list, 
    'MWU_stat':stat_list
})


In [17]:
all_results_df = all_results_df.pivot(index = ['Split', 'Direction', 'Receptor'], columns = 'Metric', values = 'MWU_stat').reset_index()

In [18]:
means_df = all_results_df.groupby(['Split', 'Receptor']).agg({'mwu_pvalue':'mean', 'mwu_stat':'mean'}).reset_index()

In [19]:

overall_means_df = means_df.groupby(['Receptor']).agg({'mwu_pvalue':'mean', 'mwu_stat':'mean'}).reset_index()


In [20]:
overall_means_df = overall_means_df.sort_values(by = 'mwu_pvalue')

In [21]:
overall_means_df_notna= overall_means_df.dropna()
q1 = np.percentile(overall_means_df_notna.mwu_pvalue, 20)
q2 = np.percentile(overall_means_df_notna.mwu_pvalue, 40)  # Median
q3 = np.percentile(overall_means_df_notna.mwu_pvalue, 60)
q4 = np.percentile(overall_means_df_notna.mwu_pvalue, 80)
q1, q2, q3

(0.00956492856035344, 0.1347336592730101, 0.4156280810582674)

In [22]:
def assign_confidence(value):
    if value < q1:
        return 'A'
    elif value < q2:
        return 'B'
    elif value < q3:
        return 'C'
    elif value < q4:
        return 'D'
    else:
        return 'E'
overall_means_df['Confidence'] = overall_means_df['mwu_pvalue'].apply(assign_confidence)

In [24]:
overall_means_df[(overall_means_df['Receptor'].str.contains('IFN')) | (overall_means_df['Receptor'].str.contains('PDCD1'))]

Metric,Receptor,mwu_pvalue,mwu_stat,Confidence
102,IFNGR2,1.415611e-09,96658.0,A
101,IFNGR1,4.422772e-08,119339.8,A
99,IFNAR1,1.024822e-01,93662.2,B
100,IFNAR2,1.025300e-01,103721.9,B
175,PDCD1,4.007491e-01,27282.4,C


In [26]:
overall_means_df.groupby('Confidence')['Receptor'].count()

Confidence
A    42
B    42
C    42
D    42
E    61
Name: Receptor, dtype: int64

In [29]:
overall_means_df.to_csv(f'results/confidence/receptor_rocauc_mean_confidence_scores_229_mwup_withna.csv')

In [30]:
overall_means_df.Confidence.unique()

array(['A', 'B', 'C', 'D', 'E'], dtype=object)

In [26]:
overall_means_df

Metric,Receptor,mwu_pvalue,mwu_stat,Confidence
179,PLD2,3.153872e-13,72249.6,A
102,IFNGR2,1.415611e-09,96658.0,A
101,IFNGR1,4.422772e-08,119339.8,A
220,TNFRSF1A,8.165990e-07,84407.0,A
59,ERBB2,4.845679e-06,360399.9,A
...,...,...,...,...
34,CD2,1.000000e+00,NaN,D
36,CD27,1.000000e+00,NaN,D
74,FSHR,1.000000e+00,NaN,D
170,NTSR1,1.000000e+00,NaN,D
